In [1]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [12]:
import argparse
import numpy as np
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import os
import json
from functools import partial
import sys
from tqdm import tqdm


from tianshou.utils import WandbLogger
from tianshou.data import Batch, Collector, ReplayBuffer, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.policy import BasePolicy, PPOPolicy
from tianshou.trainer import OnpolicyTrainer
from tianshou.utils.net.common import ActorCritic, Net
#from tianshou.utils.net.discrete import Actor, Critic
from tianshou.utils.net.continuous import Actor, Critic, ActorProb

import gymnasium as gym
from gymnasium import spaces
from gymnasium.wrappers import TimeLimit, RescaleAction, TransformObservation
from stable_baselines3.common.env_checker import check_env

from lib.environments import get_environment
from lib.environments.kolmogorov import KolmogorovEnvironment
from lib.policy import get_rl_algo
from lib.distributions import ElementwiseNormal
from lib.models import get_actor_critic
from lib.utils import str2bool, Config, dict_to_wandb_table, restrict_to_num_threads
from lib.trainer import MyOnpolicyTrainer

#temporary solution for xlb imports
sys.path.append(os.path.abspath('/home/pfischer/XLB'))
#from my_flows.kolmogorov_2d import Kolmogorov_flow
from my_flows.helpers import get_kwargs

import wandb
wandb.require("core")

device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
def get_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="CartPole-v1")
    parser.add_argument("--model", type=str, default="ppo")
    parser.add_argument("--reward_threshold", type=int, default=500)
    parser.add_argument("--seed", type=int, default=0)
    parser.add_argument("--buffer_size", type=int, default=5000)
    parser.add_argument("--max_epoch", type=int, default=20)
    parser.add_argument("--step_per_epoch", type=int, default=1000)
    parser.add_argument("--train_num", type=int, default=1)
    parser.add_argument("--test_num", type=int, default=1)
    parser.add_argument("--logdir", type=str, default="log")
    parser.add_argument("--gamma", type=float, default=0.97)
    parser.add_argument("--lr", help='learning rate', type=float, default=0.0003)
    parser.add_argument("--repeat_per_collect", type=int, default=10)
    parser.add_argument("--episode_per_test", type=int, default=1)
    parser.add_argument("--batch_size", type=int, default=32)
    parser.add_argument("--step_per_collect", type=int, default=100) 
    parser.add_argument("--architecture", type=int, default=[64, 64])
    parser.add_argument("--backbone_out_dim", type=int, default=64)

    return parser.parse_known_args()[0]


def create_env(kwargs1, kwargs2, max_t=50000, min_a=-1., max_a=1.):
    """
    creates the environemnt and applyes wrappers to action and
    observations space and sets time limit.
    """
    env = KolmogorovEnvironment(kwargs1, kwargs2)
    env = TimeLimit(env, max_episode_steps=max_t)
    env = RescaleAction(env, min_action=min_a, max_action=max_a)
    env = TransformObservation(env, lambda obs: (obs/20))
    return env


class Backbone(nn.Module):

    def __init__(self, out_size=64, device="cpu"):
        super(Backbone, self).__init__()
        
        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 2, 3, stride=2, padding=1),
            nn.BatchNorm2d(2),
            nn.ReLU(True),
            nn.Conv2d(2, 4, 3, stride=2, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(True),
            nn.Conv2d(4, 8, 3, stride=2, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            nn.MaxPool2d(2,2)
        )
        
        ### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(True),
            #nn.Dropout(p=0.5),
            nn.Linear(128, out_size),
            nn.ReLU(True),
        )
        
    #def forward(self, x):
    #    x = x.reshape(x.shape[0],1,128,128)
    #    x = self.encoder_cnn(x)
    #    x = x.reshape(x.shape[0], -1)
    #    x = self.encoder_lin(x)
    #    return x

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float, device=device)
        batch = obs.shape[0]

        obs = self.encoder_cnn(obs.reshape(batch, 1, 128, 128))
        obs = obs.reshape(batch, -1)
        logits = self.encoder_lin(obs)

        return logits, state

In [14]:
#######################################################################################################
####### setup stuff *##################################################################################
#######################################################################################################
args = get_args()
# seed
np.random.seed(args.seed)
torch.manual_seed(args.seed)

#restrict_to_num_threads(1)

In [15]:
#######################################################################################################
####### environments ##################################################################################
#######################################################################################################
u0_path = "/home/pfischer/XLB/vel_init/velocity_burn_in_1806594.npy" #4096x4096 simulation
rho0_path = "/home/pfischer/XLB/vel_init/density_burn_in_1806594.npy" #4096x4096 simulation
kwargs1, T1,_,_ = get_kwargs(u0_path=u0_path, rho0_path=rho0_path, lamb=1) #cgs 
kwargs2, T2,_,_ = get_kwargs(u0_path=u0_path, rho0_path=rho0_path, lamb=1) #fgs
#check if cgs time is a factor of fgs time
assert (T2%T1 == 0)
env = create_env(kwargs1, kwargs2, max_t=T1)
train_env = DummyVectorEnv([lambda: create_env(kwargs1, kwargs2, max_t=T1) for _ in range(args.train_num)])
test_env = DummyVectorEnv([lambda: create_env(kwargs1, kwargs2, max_t=T1) for _ in range(args.test_num)])

m_prime = 31232.0, end time = 31232 steps, T=355, io_rate = 64.0, Number of outputs = 490.0
m_prime = 31232.0, end time = 31232 steps, T=355, io_rate = 64.0, Number of outputs = 490.0


In [16]:
print(env.observation_space.shape)
print(env.action_space.shape)

(128, 128)
(1,)


In [17]:
#Policy
assert env.observation_space.shape is not None  # for mypy
assert env.action_space.shape is not None

net = Net(state_shape=env.observation_space.shape, hidden_sizes=args.architecture, device=device).to(device)
#net = Backbone(out_size=args.backbone_out_dim, device=device).to(device)
#net = Net1(state_shape=env.observation_space.shape, action_shape=(64,)).to(device)
actor = ActorProb(preprocess_net=net, action_shape=env.action_space.shape, max_action=1,
                 preprocess_net_output_dim=args.backbone_out_dim, device=device).to(device)
critic = Critic(preprocess_net=net, preprocess_net_output_dim=args.backbone_out_dim, device=device).to(device)
actor_critic = ActorCritic(actor=actor, critic=critic)
optim = torch.optim.Adam(actor_critic.parameters(), lr=args.lr)
dist = torch.distributions.Normal
policy = PPOPolicy(
    actor=actor,
    critic=critic,
    optim=optim,
    dist_fn=dist,
    action_space=env.action_space,
    deterministic_eval=True,
    action_scaling=False,
)

In [18]:
#Collectors
train_collector = Collector(
    policy=policy,
    env=train_env,
    buffer=VectorReplayBuffer(args.buffer_size, len(train_env)),
)
test_collector = Collector(policy=policy, env=test_env)
train_collector.reset()
test_collector.reset()

In [19]:
 #Trainer
trainer = OnpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=args.max_epoch,
    step_per_epoch=args.step_per_epoch,
    repeat_per_collect=args.repeat_per_collect,             
    episode_per_test=args.episode_per_test,
    batch_size=args.batch_size,
    step_per_collect=args.step_per_collect,
    #stop_fn=lambda mean_reward: mean_reward >= args.reward_threshold,
    #save_checkpoint_fn=checkpoint_fn,
    #logger=logger,
)

In [20]:
result = trainer.run()

Epoch #1: 1001it [00:06, 143.56it/s, env_step=1000, len=994, loss=30.446, loss/clip=0.005, loss/ent=1.405, loss/vf=60.911, n/ep=1, n/st=100, rew=980.59]                         


Epoch #1: test_reward: 3446.459598 ± 0.000000, best_reward: 5167.335735 ± 0.000000 in #0


Epoch #2: 1001it [00:08, 114.59it/s, env_step=2000, len=134, loss=1.352, loss/clip=-0.002, loss/ent=1.407, loss/vf=2.734, n/ep=0, n/st=100, rew=133.30]                            


Epoch #2: test_reward: 4289.824106 ± 0.000000, best_reward: 5167.335735 ± 0.000000 in #0


Epoch #3: 1001it [00:06, 154.91it/s, env_step=3000, len=994, loss=1.063, loss/clip=-0.004, loss/ent=1.425, loss/vf=2.163, n/ep=0, n/st=100, rew=980.59]                          


...

Epoch #18: test_reward: 91.482045 ± 0.000000, best_reward: 5167.335735 ± 0.000000 in #0


Epoch #19: 1001it [00:04, 242.72it/s, env_step=19000, len=92, loss=0.208, loss/clip=-0.001, loss/ent=1.540, loss/vf=0.448, n/ep=0, n/st=100, rew=91.48]                          


Epoch #19: test_reward: 91.495687 ± 0.000000, best_reward: 5167.335735 ± 0.000000 in #0


Epoch #20: 1001it [00:04, 248.18it/s, env_step=20000, len=92, loss=0.144, loss/clip=-0.001, loss/ent=1.536, loss/vf=0.321, n/ep=0, n/st=100, rew=91.48]                          


Epoch #20: test_reward: 91.481917 ± 0.000000, best_reward: 5167.335735 ± 0.000000 in #0


In [21]:
#play one episode and record statistics
reward = 0
step = 0
obs ,inf = env.reset()
policy.eval()
episode_is_over = False
#print(omg, env.unwrapped.fgs.omega, env.unwrapped.cgs.omega)
while not episode_is_over:

    action = policy(Batch(obs=np.array([obs]), info=inf)).act[0]
    obs, rew, terminated, truncated, inf = env.step(action.detach().cpu().numpy())
    #obs, rew, terminated, truncated, inf = env.step(-0.2)
    reward += rew
    step += 1
    if terminated or truncated:
        episode_is_over = True

    if step%250 == 0:
        ac = action.detach().cpu().numpy()[0]
        a = 0.98
        b = 1.02
        scaled_ac = a + 0.5*(b-a)*(ac+1)
        print(f"action = {ac}; omega_cgs = {env.unwrapped.cgs.omega}; omega2 = {env.unwrapped.fgs.omega}; rescaled action = {scaled_ac*env.unwrapped.fgs.omega}")
        env.render()

print(f"#steps = {step}, Total Reward = {reward}")


#steps = 92, Total Reward = 91.48191716757027
